# First BERT Experiments

In this notebook we do some first experiments with BERT: we finetune a BERT model+classifier on each of our datasets separately and compute the accuracy of the resulting classifier on the test data.

For these experiments we use the `pytorch_transformers` package. It contains a variety of neural network architectures for transfer learning and pretrained models, including BERT and XLNET.

Two different BERT models are relevant for our experiments: 

- BERT-base-uncased: a relatively small BERT model that should already give reasonable results,
- BERT-large-uncased: a larger model for real state-of-the-art results.

In [1]:
BERT_MODEL = 'bert-base-uncased'
BATCH_SIZE = 16 if "base" in BERT_MODEL else 2
GRADIENT_ACCUMULATION_STEPS = 1 if "base" in BERT_MODEL else 8
MAX_SEQ_LENGTH = 100
PREFIXES = ["eatingmeat4_so", "eatingmeat4_because", "eatingmeat4_but"]

## Data

We use the same data as for all our previous experiments. Here we load the training, development and test data for a particular prompt.

In [2]:
import sys
sys.path.append('../')

import ndjson
import glob
import numpy as np

from quillnlp.models.bert.preprocessing import preprocess, create_label_vocabulary

data = []
for prefix in PREFIXES:
    data_file = f"../data/interim/{prefix}_withprompt.ndjson"

    with open(data_file) as i:
        new_data = ndjson.load(i)
        for item in new_data:
            item["label"] = prefix + "_" + item["labels"][0]
        data.extend(new_data)
        
# Make sure this is a single-label problem
label_lengths = [len(item["labels"]) for item in data]
assert max(label_lengths) == 1
        
label2idx = create_label_vocabulary(data)
idx2label = {v:k for k,v in label2idx.items()}
target_names = [idx2label[s] for s in range(len(idx2label))]

data_items = preprocess(data, BERT_MODEL, label2idx, MAX_SEQ_LENGTH)
data_items = np.array(data_items)

I0406 17:49:15.544050 140397457205056 file_utils.py:41] PyTorch version 1.2.0+cu92 available.
I0406 17:49:16.530257 140397457205056 file_utils.py:57] TensorFlow version 2.1.0 available.
I0406 17:49:17.201040 140397457205056 tokenization_utils.py:501] loading file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt from cache at /home/yves/.cache/torch/transformers/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084


## Training

In [3]:
import torch
import random

from quillnlp.models.bert.train import train, evaluate
from quillnlp.models.bert.models import get_bert_classifier

from quillnlp.models.bert.preprocessing import get_data_loader
from sklearn.model_selection import KFold

random.shuffle(data_items)

kf = KFold(n_splits=5, shuffle=True, random_state=1)
all_correct, all_predicted = [], []
all_test_data = []
for train_idx, test_idx in kf.split(data_items):

    train_and_dev_data = data_items[train_idx]
    cutoff = int(len(train_and_dev_data)/4*3)
    
    train_data = train_and_dev_data[:cutoff]
    dev_data = train_and_dev_data[cutoff:]
    test_data = data_items[test_idx]

    train_dataloader = get_data_loader(train_data, BATCH_SIZE)
    dev_dataloader = get_data_loader(dev_data, BATCH_SIZE)
    test_dataloader = get_data_loader(test_data, BATCH_SIZE, shuffle=False)

    device = "cuda" if torch.cuda.is_available() else "cpu"
    model = get_bert_classifier(BERT_MODEL, len(label2idx), device=device)
    output_model_file = train(model, train_dataloader, dev_dataloader, 
                              BATCH_SIZE, GRADIENT_ACCUMULATION_STEPS, device,
                             num_train_epochs=100)
    
    print("Loading model from", output_model_file)
    device="cpu"

    model = get_bert_classifier(BERT_MODEL, len(label2idx), model_file=output_model_file, device=device)
    model.eval()
    
    _, _, test_correct, test_predicted = evaluate(model, test_dataloader, device)
    all_correct.extend(test_correct)
    all_predicted.extend(test_predicted)
    all_test_data.extend(test_data)


I0406 17:49:18.523230 140397457205056 configuration_utils.py:256] loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-config.json from cache at /home/yves/.cache/torch/transformers/4dad0251492946e18ac39290fcfe91b89d370fee250efe9521476438fe8ca185.8f56353af4a709bf5ff0fbc915d8f5b42bfff892cbb6ac98c3c45f481a03c685
I0406 17:49:18.524959 140397457205056 configuration_utils.py:292] Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": null,
  "do_sample": false,
  "eos_token_ids": null,
  "finetuning_task": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "is_decoder": false,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1
  },
  "layer_norm_eps": 1e-12,
  "length_penalty": 1.0,
  "max_length": 20,
  "max_position_embedd

../quillnlp/models/bert/train.py:146: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  outputs = softmax(logits.to('cpu'))



Loss history: []
Dev loss: 3.3455551783243815


Epoch:   1%|          | 1/100 [00:20<34:17, 20.79s/it]


Loss history: [3.3455551783243815]
Dev loss: 2.8631558179855348


...


Loss history: [3.2670257647832233, 2.903889854749044, 2.5840357065200807, 2.2129236300786337, 1.8371737043062846, 1.5553256710370382, 1.3585904439290364, 1.2083387891451518, 1.130923291047414, 1.06386499106884, 1.0118305146694184, 0.9572323580582937, 0.9083395431439082, 0.8725145826737086, 0.863960995276769, 0.8496012091636658, 0.8438244213660558, 0.8395298838615417, 0.8296309490998586, 0.8324170902371406, 0.807247390349706, 0.7803093115488688, 0.7722235937913259, 0.7673863112926483, 0.7769743020335833, 0.7831111192703247, 0.7778826514879863, 0.7511599664886792, 0.7967015504837036, 0.7773870507876078]
Dev loss: 0.7894340097904206





Epoch:  32%|███▏      | 32/100 [11:13<23:31, 20.75s/it]


Loss history: [3.2670257647832233, 2.903889854749044, 2.5840357065200807, 2.2129236300786337, 1.8371737043062846, 1.5553256710370382, 1.3585904439290364, 1.2083387891451518, 1.130923291047414, 1.06386499106884, 1.0118305146694184, 0.9572323580582937, 0.9083395431439082, 0.8725145826737086, 0.863960995276769, 0.8496012091636658, 0.8438244213660558, 0.8395298838615417, 0.8296309490998586, 0.8324170902371406, 0.807247390349706, 0.7803093115488688, 0.7722235937913259, 0.7673863112926483, 0.7769743020335833, 0.7831111192703247, 0.7778826514879863, 0.7511599664886792, 0.7967015504837036, 0.7773870507876078, 0.7894340097904206]
Dev loss: 0.7840907235940298



Loss history: [3.2670257647832233, 2.903889854749044, 2.5840357065200807, 2.2129236300786337, 1.8371737043062846, 1.5553256710370382, 1.3585904439290364, 1.2083387891451518, 1.130923291047414, 1.06386499106884, 1.0118305146694184, 0.9572323580582937, 0.9083395431439082, 0.8725145826737086, 0.863960995276769, 0.8496012091636658, 0.8438244213660558, 0.8395298838615417, 0.8296309490998586, 0.8324170902371406, 0.807247390349706, 0.7803093115488688, 0.7722235937913259, 0.7673863112926483, 0.7769743020335833, 0.7831111192703247, 0.7778826514879863, 0.7511599664886792, 0.7967015504837036, 0.7773870507876078, 0.7894340097904206, 0.7840907235940298]
Dev loss: 0.7799115518728892
No improvement on development set. Finish training.
Loading model from /tmp/model.bin


I0406 18:45:50.138985 140397457205056 configuration_utils.py:256] loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-config.json from cache at /home/yves/.cache/torch/transformers/4dad0251492946e18ac39290fcfe91b89d370fee250efe9521476438fe8ca185.8f56353af4a709bf5ff0fbc915d8f5b42bfff892cbb6ac98c3c45f481a03c685
I0406 18:45:50.140305 140397457205056 configuration_utils.py:292] Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": null,
  "do_sample": false,
  "eos_token_ids": null,
  "finetuning_task": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "is_decoder": false,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1
  },
  "layer_norm_eps": 1e-12,
  "length_penalty": 1.0,
  "max_length": 20,
  "max_position_embedd

## Evaluation

In [4]:
from sklearn.metrics import precision_recall_fscore_support, classification_report

print("Test performance:", precision_recall_fscore_support(all_correct, all_predicted, average="micro"))
print(classification_report(all_correct, all_predicted, target_names=target_names))

Test performance: (0.8082595870206489, 0.8082595870206489, 0.8082595870206488, None)
                                 precision    recall  f1-score   support

     eatingmeat4_so_Feedback_11       0.61      0.69      0.65        16
      eatingmeat4_so_Feedback_1       0.50      0.67      0.57        12
      eatingmeat4_so_Feedback_2       0.86      0.95      0.90       148
      eatingmeat4_so_Feedback_3       0.36      0.25      0.30        16
      eatingmeat4_so_Feedback_4       0.33      0.10      0.15        10
      eatingmeat4_so_Feedback_5       0.60      0.38      0.46        16
      eatingmeat4_so_Feedback_6       1.00      0.30      0.46        10
      eatingmeat4_so_Feedback_7       0.92      0.92      0.92        24
      eatingmeat4_so_Feedback_8       0.82      0.48      0.61        48
      eatingmeat4_so_Feedback_9       0.85      0.89      0.87       309
     eatingmeat4_so_Feedback_10       0.86      0.90      0.88       241
 eatingmeat4_because_Feedback_1       

/opt/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [5]:
c = 0
for item, predicted, correct in zip(all_test_data, all_predicted, all_correct):
    assert item.label_ids == correct
    c += (item.label_ids == predicted)
    print("{}#{}#{}".format(item.text, idx2label[correct], idx2label[predicted]))
    
print()
print(c, "/", len(all_test_data), "=", c/len(all_test_data))

Methane from cow burps harms the environment, so either people have to stop eating meat and use plant based "meats" that don't harm the environment as much#eatingmeat4_so_Feedback_9#eatingmeat4_so_Feedback_9
Methane from cow burps harms the environment, but they only produce about 14.5% of greenhouse gas emissions.#eatingmeat4_but_Feedback_1#eatingmeat4_but_Feedback_1
Methane from cow burps harms the environment, but impossible Foods have created plant based meat that have the same qualities as real meat.#eatingmeat4_but_Feedback_8#eatingmeat4_but_Feedback_5
Methane from cow burps harms the environment, but when cows eat seaweed as part of their diet, the methane content in their burps decreases by 99%.#eatingmeat4_but_Feedback_13#eatingmeat4_but_Feedback_13
Methane from cow burps harms the environment because it exacerbates the process of global warming by increasing Earth's temperature.#eatingmeat4_because_Feedback_1#eatingmeat4_because_Feedback_1
Methane from cow burps harms the env